# Load Disaggregation using Factorial Hidden Markov Model (FHMM)
The london data that was inputed to peform simple load disaggregation from https://github.com/amzkit/load-disaggregation, First things was the preprocess the data for disaggregation.

In [1]:
import fhmm_model as fhmm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib as mpl
from matplotlib.ticker import FuncFormatter

In [2]:
df = pd.read_csv("london_sample.csv")
df = df.drop(columns='Unnamed: 0')
df.columns = ['LCLid', 'DateTime', 'KWh']
print(df)
dfdata = pd.read_csv("fake_appliance_data.csv")
print(dfdata)

             LCLid                   DateTime        KWh
0        MAC000323  2012-03-06 14:00:00+00:00   488.0000
1        MAC000323  2012-03-06 14:30:00+00:00   449.0000
2        MAC000323  2012-03-06 15:00:00+00:00   424.0000
3        MAC000323  2012-03-06 15:30:00+00:00   439.0000
4        MAC000323  2012-03-06 16:00:00+00:00   291.0000
...            ...                        ...        ...
7270639  MAC005509  2014-02-27 22:00:00+00:00   697.0000
7270640  MAC005509  2014-02-27 22:30:00+00:00  1496.0001
7270641  MAC005509  2014-02-27 23:00:00+00:00  1092.0000
7270642  MAC005509  2014-02-27 23:30:00+00:00  1002.0000
7270643  MAC005509  2014-02-28 00:00:00+00:00   282.0000

[7270644 rows x 3 columns]
                      DateTime      Fridge  Washing Machine          TV  \
0    2013-01-05 07:40:11+00:00  140.717578      1199.616497  268.105619   
1    2013-06-09 01:03:07+00:00  160.390627      1328.789888  226.854459   
2    2012-07-13 10:57:19+00:00  117.402343       999.973534  17

In [3]:
df['DateTime'] = pd.to_datetime(df['DateTime'])
df.set_index('DateTime', inplace=True)

dfdata['DateTime'] = pd.to_datetime(dfdata['DateTime'])
dfdata.set_index('DateTime', inplace=True)

merged_df = df.join(dfdata)

merged_df.dropna(inplace=True)
print(merged_df)

Empty DataFrame
Columns: [LCLid, KWh, Fridge, Washing Machine, TV, Oven, Air Conditioner]
Index: []


Using the FHMM models I needed to change the column names to names that would work with models.  

In [4]:
split_index = int(len(merged_df) * 0.75)
train_df = merged_df.iloc[:split_index]
test_df = merged_df.iloc[split_index:]

train_df = train_df.reset_index()
train_df['timestamp'] = train_df['DateTime'].astype(int) // 10**9
train_df = train_df[['timestamp', 'KWh', 'Fridge', 'Washing Machine', 'TV', 'Oven', 'Air Conditioner']]
train_df = train_df.rename(columns={'KWh': 'power'})
print(train_df)
test_df = test_df.reset_index()
test_df['timestamp'] = test_df['DateTime'].astype(int) // 10**9
test_df = test_df[['timestamp', 'KWh']]
test_df = test_df.rename(columns={'KWh': 'power'})
print(test_df)

Empty DataFrame
Columns: [timestamp, power, Fridge, Washing Machine, TV, Oven, Air Conditioner]
Index: []
Empty DataFrame
Columns: [timestamp, power]
Index: []


In [5]:
# Create a list of appliances
list_of_appliances = ['Fridge', 'Washing Machine', 'TV', 'Oven', 'Air Conditioner']

# Create an instance of the FHMM model
model = fhmm.FHMM()

# Train the model with the training data
model.train(train_df, list_of_appliances)

model.save("fhmm_trained_model.pkl")

 [train] ERROR Submeter 'Fridge' has no samples, skipping...
 [train] ERROR Submeter 'Washing Machine' has no samples, skipping...
 [train] ERROR Submeter 'TV' has no samples, skipping...
 [train] ERROR Submeter 'Oven' has no samples, skipping...
 [train] ERROR Submeter 'Air Conditioner' has no samples, skipping...


IndexError: list index out of range

In [ ]:
# Disaggregate the loads using the testing data
prediction = model.disaggregate(test_df)
print(prediction)
# Create a bar chart of the original data
fig, ax = plt.subplots(figsize=(12, 6))
print(df)
mpl.rcParams['agg.path.chunksize'] = 100000
# Plot the original energy consumption data
ax.plot(df.index, df['KWh']*1000, color='gray')
